Rogelio Jesús Villarreal De Ochoa
A00838563 - 15 de agosto de 2025

# Explicación de código
Rogelio Jesús Villarreal De Ochoa A00838563

Para entender cómo adaptamos el agente para cumplir la función de recoger la tierra, empezamos desde el modelo base de los agentes que vimos en clase. Usando ese modelo, podemos agregar comportamientos personalizados, para que en nuestro caso, busque y limpie tierra.

Espero que la entrega cumpla con los requerimientos, habíamos discutido que lo importante era el rastreo y limpieza de tierra, por lo que fue la meta final de nuestro progama.

**Importamos lo siguiente de nuestro trabajo previo en clase:**
- bibliotecas -> agentpy, numpy(random) matplotlib, IPython.display
- my_plot
- DummyAgent
- DummyModel

Como base, tenemos un agente que se mueve en una dirección. Vamos a modificar y agregar funciones.

### Class DummyModel

**setup:**
Definimos height y width como dimensiones de matriz para la generación de tierra.

Para representar a los lugares con tierra, hicimos una matríz copia, agregando dos tierras iniciales.

También almacenaremos todas las coordenadas en una lista como tuplas, para identificar dónde estan los lugares sucios

**getDirtDirection:**
Tomando la posición actual del agente, necesitamos determinar la coordenada sucia más cercana al agente.

Para eso, revisamos la lista de coordenadas sucias, obteniendo la más cercana.

**roombaMove:**
Llamando getDirtDirection(), obtenemos la diferencia entre pos actual y coordenada más cercana. Regresa 0, 1 o -1 dependiendo de la diferencia en ambos ejes. Esto se pone en self.execute como comportamiento de movimiento.

**generateRandomDirt:**
Usando random y dimensiones de la grid. Se agregan los valores resultantes como 1 en la cuadricula, y en dirty_coords.

**cleanDirt:**
Simplemente asigna como 0 cuando el agente esta en cima del cuadrado.

**step:**
Se ordenaron los comandos para que primero limpie la tierra -> se mueva -> genere tierra aleatoria. Causa que se detenga un tick si no hay tierra, pero se puede personalizar.



In [7]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import HTML
import agentpy as ap


In [ ]:
def my_plot(model, ax):
    grid = np.zeros(model.environment.shape)
    print(model.environment.positions)
    for agent, pos in model.environment.positions.items():
        grid[pos] = agent.id
    # Agregamos valores para dirt en la grid para poder visualizarlo
    for coord in model.dirty_coords:
        grid[coord] = 1
    #sns.heatmap (ax=ax, grid, annot=True)
    ax.imshow(grid, cmap='Greys')

'''
simple agent single direction
'''

class DummyAgent(ap.Agent):
    def setup(self, dir):
        # Agent's position
        self.direction = dir
    
    def execute(self):
        # Agents will update their position
        self.direction = self.model.roombaMove()
        self.model.environment.move_by(self, self.direction)

'''
simple glass modeling the MAS containing DummyAgent
'''

class DummyModel(ap.Model):
    def setup(self):
        #System = ((Ag1, Ag2), Env, ())
        self.width = 7
        self.height = 7
        self.environment = ap.Grid(self, (self.width, self.height))
        
        # Dirtycoords
        self.dirty_grid = np.zeros(self.environment.shape, dtype=int)
        self.dirty_grid[3,3] = 1
        self.dirty_grid[0, 4] = 1
        
        self.dirty_coords = []
        self.dirty_coords.append((3,3))
        self.dirty_coords.append((0,4))
        
        # Crear el agente
        self.roomba_agent = DummyAgent(self, (0,0))
        self.environment.add_agents([self.roomba_agent], positions=[(0,0)])
        self.roomba_agent.setup((0,0))

    
    def getDirtDirection(self):
        currentpos = self.environment.positions[self.roomba_agent]
        minDistance = float('inf')
        
        # Si no encuentra tierra, regresa su misma pos
        closest = currentpos
        
        for coord in self.dirty_coords:
            # Dist absoluta
            xDistance = abs(coord[0] - currentpos[0])
            yDistance = abs(coord[1] - currentpos[1])
            distance = xDistance + yDistance
            # Actualizamos minDistance para encontrar la coord mas cercana
            if distance < minDistance:
                minDistance = distance
                closest = coord
        return closest
    
    def roombaMove(self):
        currentpos = self.environment.positions[self.roomba_agent]
        
        if not self.dirty_coords:
            return(0,0)
        
        target = self.getDirtDirection()
        
        xDir = target[0] - currentpos[0]
        yDir = target[1] - currentpos[1]
        
        dx = 0 if xDir == 0 else (1 if xDir > 0 else -1)
        dy = 0 if yDir == 0 else (1 if yDir > 0 else -1)
        
        return (dx, dy)
    
    def execute(self):
        # Agents will update their position
        self.model.environment.move_by(self, self.direction)
    
    def generateRandomDirt(self):
        randX = np.random.randint(0,self.width)
        randY = np.random.randint(0,self.height)
        
        # Check duplicados por si queremos generar mas de 1 tierra a la vez
        if (randX, randY) not in self.dirty_coords:
            self.dirty_grid[randX, randY] = 1
            self.dirty_coords.append((randX, randY))
        
    def cleanDirt(self):
        currentpos = self.environment.positions[self.roomba_agent]
        if self.dirty_grid[currentpos]:
            self.dirty_grid[currentpos] = 0
            # Check por si las dudas no quitar nada que no está
            if currentpos in self.dirty_coords:
                self.dirty_coords.remove(currentpos)

    def step(self):
        if self.p.print:
            print("**************\nRight agent positions:{}".format(self.environment.positions[self.roomba_agent]))
        #On each step agents move in a given direction
        self.cleanDirt()
        self.environment.agents.execute()
        if not self.dirty_coords:
            self.generateRandomDirt()

fig, ax = plt.subplots()
parameters = {'print': True, 'steps': 50}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())


{DummyAgent (Obj 2): (0, 0)}
{DummyAgent (Obj 2): (0, 0)}
**************
Right agent positions:(0, 0)
{DummyAgent (Obj 2): (0, 1)}
**************
Right agent positions:(0, 1)
{DummyAgent (Obj 2): (0, 2)}
**************
Right agent positions:(0, 2)
{DummyAgent (Obj 2): (0, 3)}
**************
Right agent positions:(0, 3)
{DummyAgent (Obj 2): (0, 4)}
**************
Right agent positions:(0, 4)
{DummyAgent (Obj 2): (1, 3)}
**************
Right agent positions:(1, 3)
{DummyAgent (Obj 2): (2, 3)}
**************
Right agent positions:(2, 3)
{DummyAgent (Obj 2): (3, 3)}
**************
Right agent positions:(3, 3)
{DummyAgent (Obj 2): (3, 3)}
**************
Right agent positions:(3, 3)
{DummyAgent (Obj 2): (4, 4)}
**************
Right agent positions:(4, 4)
{DummyAgent (Obj 2): (5, 4)}
**************
Right agent positions:(5, 4)
{DummyAgent (Obj 2): (6, 4)}
**************
Right agent positions:(6, 4)
{DummyAgent (Obj 2): (6, 4)}
**************
Right agent positions:(6, 4)
{DummyAgent (Obj 2): (